### Kaggle BaseLine

In [1]:
#analyse
import pandas as pd
import numpy as np


# prepare data
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.utils import shuffle


#processing
from sklearn.pipeline import Pipeline


#Quality control and metrics
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score


#Models
import xgboost as xgb
from xgboost import XGBClassifier
from catboost import Pool, CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection,linear_model, preprocessing, ensemble, metrics, svm
from sklearn.linear_model import LogisticRegression
import lightgbm


#scaller
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer

#encoder
import category_encoders as ce
from category_encoders.cat_boost import CatBoostEncoder


#fill nan
from sklearn.impute import SimpleImputer


## visualise
import seaborn as sns


#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
test_data = pd.read_csv('/Users/nikli/Documents/git_reps/Data/orange_small_churn_test_data.csv',index_col = 'ID')
train_data = pd.read_csv('/Users/nikli/Documents/git_reps/Data/orange_small_churn_train_data.csv',index_col = 'ID')
train_labels=train_data['labels'].fillna(-1)
train_data['labels']=train_data['labels'].fillna(-1)
train_data=train_data.drop('labels', axis=1)
train_data.loc[:,'Var190':].head()

,Var190,Var191,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
ID,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NESt0G8EIb,AERks4l,NaN,taul,1K8T,0LaQ,UaKK0yW,I1sFbv_0IT,...,Al6ZaUT,vr93T2a,LM8l689qOp,NaN,NaN,fKCe,02N6s8f,xwM2aC7IdeMC0,NaN,NaN
1,NaN,NaN,P1WvyxLp3Z,2Knk1KF,NaN,taul,1K8T,YFAj,Bnunsla,o64y9zI,...,oslk,6hQ9lNX,LM8l689qOp,NaN,ELof,xb3V,RAYp,55YFVY9,mj86,NaN
2,NaN,NaN,FoxgUHSK8h,LrdZy8QqgUfkVShG,NaN,taul,1K8T,TyGl,fhk21Ss,nQUveAzAF7,...,zCkv,catzS2D,LM8l689qOp,NaN,NaN,FSa2,ZI9m,ib5G6X1eUxUn6,mj86,NaN
3,NaN,NaN,vNEvyxLp3Z,RO12,NaN,taul,1K8T,0Xwj,uoZk2Zj,LWyxgtXeJL,...,oslk,e4lqvY0,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN
4,NaN,NaN,4e7gUH7IEC,RO12,NaN,taul,1K8T,vSNn,kugYdIL,ZIXKpoNpqq,...,oslk,MAz3HNj,LM8l689qOp,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN,NaN


## План:
Построить базовые модели на основе преобразования датасета различными способами и замерить качество базовых моделей:
<!-- - <a href='#fillna'> 1. Закодировать категориальные признаки,заменить пропуски </a>
- <a href='#first_result'> 2. Сделать просто фит предикт - записать результат модели на сабмите </a>
- <a href='#prepare_dataset'> 3. Сделать анализ датафрейма, удалить признаки, которые слабо коррелируют - записать результат модели на сабмите </a>
- <a href='#prepare_dataset'> 6. Сделать кросс валидацию & подбор параметров модели - записать результат модели на сабмите  </a>
- <a href='#prepare_dataset'> 7. Сделать вывод про лучший скор </a>
 -->

#### Удалим столбцы тестовой выборки,в которых нет никакой информации : set(column) = {Nan}

In [3]:
print('Способ поиска пропусков')
print(train_data['Var79'].unique())
print('Так мы будем находить массив уникальные значения ,которые являются рпопусками или нет')
print([np.isnan(nan) for nan in train_data['Var79'].unique()])
print('Проверим на наличие пропусков')
print(True in [np.isnan(nan) for nan in train_data['Var79'].unique()])
array=[np.isnan(nan) for nan in train_data['Var78'].unique()]
print('Мы должны добавить ещё одну проверку , на длинну массива')
print(array)
print(False in array)

Способ поиска пропусков
[nan]
Так мы будем находить массив уникальные значения ,которые являются рпопусками или нет
[True]
Проверим на наличие пропусков
True
Мы должны добавить ещё одну проверку , на длинну массива
[False, True, False, False, False, False, False, False, False, False, False, False, False]
True


#### Удалим неинформативные признаки

In [5]:
def undersampling(data, labels,col_to_label):
    from sklearn.utils import shuffle

    data['Labels']=labels
    dataChurn = data[data['Labels'] == 1]
    
    dataNoChurn = data[data['Labels'] == -1]
    
    countFirst = len(dataChurn)
    dataChurn = dataChurn.append(dataNoChurn.iloc[:countFirst,:])
    dataChurn = shuffle(dataChurn)
    return dataChurn.iloc[:,:col_to_label], dataChurn.iloc[:,col_to_label:]
train_data , train_labels = undersampling(train_data, train_labels, 230)

def dropUninformative(data):
    data=data.copy()
    df = pd.DataFrame([])
    for i in range(data.shape[1]):
        if len(data.iloc[:,i].unique())>1:
            df["Var"+str(i+1)] = data.iloc[:,i]
    return df

train_data=dropUninformative(train_data)
test_data=test_data[train_data.columns]


In [7]:
### Теперь мы можем тестировать способы замены пропусков в числах:
### Теперь мы можем тестировать способы кодирования данных:
def fill_numeric(train_data,  strategy="mean"):
    train_data=train_data.copy()
    if strategy=="mean":
        imputer = SimpleImputer(missing_values=np.nan, strategy=strategy)

    if strategy=='median':
        imputer = SimpleImputer(missing_values=np.nan, strategy='median')

    if strategy=='most_frequent':
        imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    if strategy=='0':
        train_data=train_data.fillna(0)
        
#         y=train_data['labels'] 
#         x=train_data.drop('labels', axis=1)

        ## Делим на обучение и тест
#         X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1234)

#         model = LogisticRegression()

#         model.fit(X_train, y_train)

#         preds_class = model.predict(X_test)

#         f1 = f1_score(y_test, preds_class, average='macro')

        return train_data


    train_data.loc[:,:] = imputer.fit_transform(train_data.loc[:,:])
    
#     y=train_data['labels'] 
#     x=train_data.drop('labels', axis=1)

#     ## Делим на обучение и тест
#     X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1234)
    
#     model = LogisticRegression()

#     model.fit(X_train, y_train)
    
#     preds_class = model.predict(X_test)
    
#     f1 = f1_score(y_test, preds_class, average='macro')
    
#     print(f1)
    return train_data
train_data['labels']=train_labels
fill_numeric(train_data[train_data.columns.difference(train_data.loc[:,'Var191':'Var229'].columns)],  strategy="mean")
fill_numeric(train_data[train_data.columns.difference(train_data.loc[:,'Var191':'Var229'].columns)],  strategy="median")
fill_numeric(train_data[train_data.columns.difference(train_data.loc[:,'Var191':'Var229'].columns)],  strategy="most_frequent")
fill_numeric(train_data[train_data.columns.difference(train_data.loc[:,'Var191':'Var229'].columns)],  strategy="0")


,Var1,Var10,Var100,Var101,Var102,Var103,Var104,Var105,Var106,Var107,...,Var91,Var92,Var93,Var94,Var95,Var96,Var97,Var98,Var99,labels
ID,,,,,,,,,,,,,,,,,,,,,
536,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,18510.0,0.0,0.0,0.0,0.0,0.0,-1.0
14272,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
413,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,494286.0,0.0,0.0,0.0,0.0,0.0,1.0
1459,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,171759.0,0.0,0.0,0.0,0.0,0.0,-1.0
1037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,254985.0,0.0,0.0,0.0,0.0,0.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1088,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3513.0,0.0,0.0,0.0,0.0,0.0,-1.0
7685,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,44943.0,0.0,0.0,0.0,0.0,0.0,1.0
5282,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [8]:
### Теперь мы можем тестировать способы замены пропусков в числах:
### Теперь мы можем тестировать способы кодирования данных:
def fill_cat(train_data,  strategy="most_frequent"):
    train_data=train_data.copy()
    
    if strategy=='most_frequent':
        imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        train_data.loc[:,:] = imputer.fit_transform(train_data.loc[:,:])
        print('kek')
    if strategy=='No_info':
        train_data=train_data.fillna('No_info')
        print('Done No_info!')
    
    if strategy=='bfill':
        train_data=train_data.fillna(method='bfill')
        train_data=train_data.fillna('No_info')
        print('Done method bfill!')
    if strategy=='pad':
        train_data=train_data.fillna(method='pad')
        train_data=train_data.fillna('No_info')
        print('Done method pad!')    


    return train_data
train_data['labels']=train_labels


fill_cat(train_data[train_data.columns.difference(train_data.loc[:,'Var1':'Var190'].columns)],  strategy="most_frequent")
fill_cat(train_data[train_data.columns.difference(train_data.loc[:,'Var1':'Var190'].columns)],  strategy="No_info")
fill_cat(train_data[train_data.columns.difference(train_data.loc[:,'Var1':'Var190'].columns)],  strategy="bfill")
fill_cat(train_data[train_data.columns.difference(train_data.loc[:,'Var1':'Var190'].columns)],  strategy="pad")


kek
Done No_info!
Done method bfill!
Done method pad!


,Var191,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,labels
ID,,,,,,,,,,,,,,,,,,,,,
536,No_info,xOXrjIQq3d,RO12,SEuy,taul,1K8T,Ti2P,H5fu_5h,pwC_jOAMsP,CdD4sos,...,oslk,LiU3odP,LM8l689qOp,No_info,kG3k,FSa2,RAYp,F2FyR07IdsN7I,No_info,-1.0
14272,No_info,m9SrEy7Rm6,g62hiBSaKg,SEuy,LfvqpCtLOY,1K8T,sKnY,fhk21Ss,UFIdC8I,CdD4sos,...,oslk,catzS2D,jySVZNlOJy,No_info,kG3k,Aoh3,ZI9m,ib5G6X1eUxUn6,mj86,1.0
413,No_info,8I1q9ayE15,RO12,SEuy,taul,1K8T,c9jp,B5n4Lhq,V6XUlxabJ4VaD,CdD4sos,...,oslk,ghXQowC,LM8l689qOp,No_info,kG3k,TNEC,RAYp,F2FyR07IdsN7I,mj86,1.0
1459,No_info,vAsvyx7ejg,RO12,SEuy,taul,1K8T,lK27,Te2Vp7s,uAJBBe1,CdD4sos,...,oslk,cwvTzYj,LM8l689qOp,No_info,kG3k,szEZ,RAYp,F2FyR07IdsN7I,mj86,-1.0
1037,No_info,DHevyx7IEC,2Knk1KF,SEuy,taul,1K8T,487l,4kbxW2B,k10MzgT,oKTwCgJ,...,oslk,BfxYyp5,LM8l689qOp,No_info,ELof,szEZ,RAYp,55YFVY9,mj86,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1088,r__I,l7Fq9ayE15,2Knk1KF,SEuy,taul,1K8T,TyGl,47W4ewH,n1zVHpT8NN,jYfhcsI,...,oslk,TRNQv5o,LM8l689qOp,4n2X,kG3k,7P5s,RAYp,TCU50_Yjmm6GIBZ0lL_,am7c,-1.0
7685,r__I,2jirEyXktW,RO12,SEuy,taul,1K8T,hoa9,hWvno_n,glRBFJT8NN,jYfhcsI,...,oslk,A5pE4Qn,LM8l689qOp,4n2X,kG3k,Aoh3,RAYp,F2FyR07IdsN7I,am7c,1.0
5282,r__I,6m0t0G8rTN,RO12,SEuy,taul,1K8T,NF63,7OKCyD5,wSIbcWv,jYfhcsI,...,oslk,bM5Esaq,LM8l689qOp,4n2X,kG3k,fKCe,RAYp,F2FyR07IdsN7I,am7c,1.0


#### Закодируем данные

In [9]:
def cat_coding(train_data, method):
    train_data=train_data.copy()
    cols=train_data.loc[:,'Var191':'Var229'].columns
    
    if method=='frequency':
        for col in cols:
            freq_encode = train_data.groupby(col).size() / train_data.shape[0]
            train_data.loc[:, col] = train_data[col].map(freq_encode)
        train_data= train_data
        
    elif method=='mean':
        for col in cols:
            mean_encode = train_data.groupby(col)['labels'].mean() # Считаем среднее значение таргета группиря по нашему столбцу
            train_data.loc[:, col] = train_data[col].map(mean_encode)
        train_data= train_data 
        
    elif method=='smoothed':
        for col in cols:
            mean = train_data['labels'].mean() # Находим среднее значение
            agg = train_data.groupby(col)['labels'].agg(['count', 'mean']) # Считаем среднее значение и число повторов значения столбца
            counts = agg['count']
            means = agg['mean']
            weight = 100
            smooth = (counts * means + weight * mean) / (counts + weight)
            train_data.loc[:, col] = train_data[col].map(smooth)
            train_data= train_data
            
    elif method=='WOE':#Weight of Evidence encoding | считаем ln отношения вероятностей принадлежности к классам
        WOEE_encoder = ce.WOEEncoder()
        print(train_data.isna().sum().sum())
        print(np.any(np.isnan(train_data)))
        train_data.loc[:,'Var191':'Var229'] = WOEE_encoder.fit_transform(train_data.loc[:,'Var191':'Var229'],
                                                                         train_data['labels'])
        train_data= train_data
    elif method=='leave_one_out':#Leave-one-out encoding | Считает среднее значение таргета для случая, когда пример удален из дата-сета.
        LOOE_encoder = ce.LeaveOneOutEncoder()
        train_data.loc[:,'Var191':'Var229'] = LOOE_encoder.fit_transform(train_data.loc[:,'Var191':'Var229'],
                                                                         train_data['labels'])
        train_data=train_data

    elif method=='сatboost':
        CBE_encoder = CatBoostEncoder()
        train_data.loc[:,'Var191':'Var229'] = CBE_encoder.fit_transform(train_data.loc[:,'Var191':'Var229'],
                                                                         train_data['labels'])
        train_data= train_data
#     y=train_data['Labels'] 
#     x=train_data.drop('Labels', axis=1)
#     ## Делим на обучение и тест
#     X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1234)
#     model = LogisticRegression()
#     model.fit(X_train, y_train)
#     preds_class = model.predict(X_test)
#     f1 = f1_score(y_test, preds_class, average='macro')
#     print(f1)
#     print('Done! ' + method)
    pass
    return train_data
        
cat_coding(train_data,  method="frequency")
cat_coding(train_data,  method="mean")
cat_coding(train_data,  method="smoothed")
cat_coding(train_data,  method="leave_one_out")
cat_coding(train_data,  method="сatboost")


,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,labels
ID,,,,,,,,,,,,,,,,,,,,,
536,NaN,NaN,NaN,NaN,NaN,1071.0,7.0,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.0
14272,NaN,NaN,NaN,NaN,NaN,777.0,28.0,NaN,NaN,NaN,...,-0.500000,0.000000,0.000000,-0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
413,NaN,NaN,NaN,NaN,NaN,77.0,0.0,NaN,NaN,NaN,...,0.000000,0.000000,-0.500000,0.000000,0.500000,0.000000,-0.500000,-0.500000,-0.500000,1.0
1459,NaN,NaN,NaN,NaN,NaN,490.0,7.0,NaN,NaN,NaN,...,0.250000,0.000000,0.000000,0.250000,0.666667,0.000000,0.000000,0.000000,0.000000,-1.0
1037,NaN,NaN,NaN,NaN,NaN,2730.0,7.0,NaN,NaN,NaN,...,0.000000,0.000000,-0.250000,0.000000,0.000000,-0.500000,-0.250000,0.000000,-0.250000,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1088,NaN,NaN,NaN,NaN,NaN,266.0,0.0,NaN,NaN,NaN,...,0.048502,0.000000,0.006969,0.003700,-0.073214,-0.198473,0.061759,-0.276923,-0.155357,-1.0
7685,NaN,NaN,NaN,NaN,NaN,399.0,0.0,NaN,NaN,NaN,...,0.048004,0.285714,0.006468,0.003328,0.118188,0.043796,0.061235,0.093145,0.108657,1.0
5282,NaN,NaN,NaN,NaN,NaN,196.0,0.0,NaN,NaN,NaN,...,0.048456,0.250000,0.006780,0.003697,0.118766,0.053691,0.061698,0.093619,0.109189,1.0


In [42]:
def results_append (model : str, cat_cod : str, fill_num : str, fill_cat : str,  data, labels):
    print(model)
    if model=='CatBoost':
        clf=CatBoostClassifier(iterations=90)
    elif model=='XGBoost':
        clf=XGBClassifier()
    elif model=='Logistsic Regression':
        print('deeed')
        clf=LogisticRegression()
    elif model=='LightGBM':
        clf=lightgbm.LGBMClassifier()
    elif model=='Decision Tree':
        clf=DecisionTreeClassifier()
    elif model=='Random Forest':
        clf=ensemble.RandomForestClassifier()
        
#     data=data.copy()
#     y = data['labels']
#     X = data.drop('labels', axis=1)
    
    X = data
    y =labels
    print(X.columns)
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12345678)
    #clf=LogisticRegression()
    print(clf)
    clf.fit(x_train, y_train)
    globals()['result3'] = globals()['result3'].append({
        'Model' : model, 
        'Cat_coding' : cat_cod,
        'Num_fill' : fill_num,
        'Cat_fill' : fill_cat,
        'Train auc score' : round(roc_auc_score(y_train, clf.predict_proba(x_train)[:, 1]), 3), 
        'Train f1 score' : round(f1_score(y_train, clf.predict(x_train)), 3),
        'Test auc score' : round(roc_auc_score(y_test, clf.predict_proba(x_test)[:, 1]), 3), 
        'Test f1 score' : round(f1_score(y_test, clf.predict(x_test)), 3)},
        ignore_index=True)

#     print(x_train.shape)
#     print(x_test.shape)
    
#     print('Train auc score' ,round(roc_auc_score(y_train, clf.predict_proba(x_train)[:, 1]), 3))
#     print('Train f1 score' , round(f1_score(y_train, clf.predict(x_train)), 3))
#     print('Test auc score' , round(roc_auc_score(y_test, clf.predict_proba(x_test)[:, 1]), 3))
#     print('Test f1 score' ,round(f1_score(y_test, clf.predict(x_test)), 3))

#     print(y_train)
#     print(y_train.value_counts())
#     print(clf.predict(x_train))
    

In [43]:
cat_fill_methods=['most_frequent', 'No_info', 'bfill', 'pad']
num_fill_methods=['mean', 'median', 'most_frequent', '0']
cat_code_methods=['frequency', 'mean', 'smoothed', 'leave_one_out', 'сatboost']
models=['CatBoost', 'XGBoost', 'Logistsic Regression', 'LightGBM', 'Decision Tree', 'Random Forest']

In [44]:
result3 = pd.DataFrame({'Model' : [], 'Cat_coding':[], 'Num_fill':[], 'Cat_fill':[], 'Train auc score' : [],
                       'Train f1 score' : [], 'Test auc score' : [], 'Test f1 score' : []})
result3

,Model,Cat_coding,Num_fill,Cat_fill,Train auc score,Train f1 score,Test auc score,Test f1 score


In [46]:
for cat_meth in cat_fill_methods:
    for num_meth in num_fill_methods:
        for cat_cod_meth in cat_code_methods:
            for model in models:
                train_data_exp = train_data.copy()
                
                train_data_exp[train_data_exp.columns.difference(train_data_exp.loc[:,'Var191':'Var229'].columns)]=fill_numeric(
                    train_data_exp[train_data_exp.columns.difference(train_data_exp.loc[:,'Var191':'Var229'].columns)],
                    strategy=num_meth)

                train_data_exp[train_data_exp.columns.difference(train_data_exp.loc[:,'Var1':'Var190'].columns)]=fill_cat(
                    train_data_exp[train_data_exp.columns.difference(train_data_exp.loc[:,'Var1':'Var190'].columns)],
                    strategy=cat_meth)

                train_data_exp=cat_coding(train_data_exp,  method=cat_cod_meth)
        
                print(model,cat_cod_meth,num_meth,cat_meth)
                labels=train_data_exp['labels']
                del train_data_exp['labels']
                results_append (model=model, cat_cod=cat_cod_meth, fill_num=num_meth,
                                fill_cat=cat_meth, data=train_data_exp, labels=labels)
                

kek
LightGBM leave_one_out mean most_frequent
LightGBM
Index(['Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6', 'Var7', 'Var9', 'Var10',
       'Var11',
       ...
       'Var220', 'Var221', 'Var222', 'Var223', 'Var224', 'Var225', 'Var226',
       'Var227', 'Var228', 'Var229'],
      dtype='object', length=212)
LGBMClassifier()
kek
Decision Tree leave_one_out mean most_frequent
Decision Tree
Index(['Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6', 'Var7', 'Var9', 'Var10',
       'Var11',
       ...
       'Var220', 'Var221', 'Var222', 'Var223', 'Var224', 'Var225', 'Var226',
       'Var227', 'Var228', 'Var229'],
      dtype='object', length=212)
DecisionTreeClassifier()
kek
Random Forest leave_one_out mean most_frequent
Random Forest
Index(['Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6', 'Var7', 'Var9', 'Var10',
       'Var11',
       ...
       'Var220', 'Var221', 'Var222', 'Var223', 'Var224', 'Var225', 'Var226',
       'Var227', 'Var228', 'Var229'],
      dtype='object', length=212)
Rando

LightGBM leave_one_out median No_info
LightGBM
Index(['Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6', 'Var7', 'Var9', 'Var10',
       'Var11',
       ...
       'Var220', 'Var221', 'Var222', 'Var223', 'Var224', 'Var225', 'Var226',
       'Var227', 'Var228', 'Var229'],
      dtype='object', length=212)
LGBMClassifier()
Done No_info!
Decision Tree leave_one_out median No_info
Decision Tree
Index(['Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6', 'Var7', 'Var9', 'Var10',
       'Var11',
       ...
       'Var220', 'Var221', 'Var222', 'Var223', 'Var224', 'Var225', 'Var226',
       'Var227', 'Var228', 'Var229'],
      dtype='object', length=212)
DecisionTreeClassifier()
Done No_info!
Random Forest leave_one_out median No_info
Random Forest
Index(['Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6', 'Var7', 'Var9', 'Var10',
       'Var11',
       ...
       'Var220', 'Var221', 'Var222', 'Var223', 'Var224', 'Var225', 'Var226',
       'Var227', 'Var228', 'Var229'],
      dtype='object', length=212)
R

Done method bfill!
LightGBM leave_one_out most_frequent bfill
LightGBM
Index(['Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6', 'Var7', 'Var9', 'Var10',
       'Var11',
       ...
       'Var220', 'Var221', 'Var222', 'Var223', 'Var224', 'Var225', 'Var226',
       'Var227', 'Var228', 'Var229'],
      dtype='object', length=212)
LGBMClassifier()
Done method bfill!
Decision Tree leave_one_out most_frequent bfill
Decision Tree
Index(['Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6', 'Var7', 'Var9', 'Var10',
       'Var11',
       ...
       'Var220', 'Var221', 'Var222', 'Var223', 'Var224', 'Var225', 'Var226',
       'Var227', 'Var228', 'Var229'],
      dtype='object', length=212)
DecisionTreeClassifier()
Done method bfill!
Random Forest leave_one_out most_frequent bfill
Random Forest
Index(['Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6', 'Var7', 'Var9', 'Var10',
       'Var11',
       ...
       'Var220', 'Var221', 'Var222', 'Var223', 'Var224', 'Var225', 'Var226',
       'Var227', 'Var228', 'V

In [54]:
result3[result3['Model']=='LightGBM'].sort_values(by=['Train f1 score', 'Test auc score','Test f1 score'],ascending=False ).head(12)

,Model,Cat_coding,Num_fill,Cat_fill,Train auc score,Train f1 score,Test auc score,Test f1 score
0,LightGBM,leave_one_out,mean,most_frequent,1.0,1.0,1.000,1.000
6,LightGBM,leave_one_out,median,most_frequent,1.0,1.0,1.000,1.000
12,LightGBM,leave_one_out,most_frequent,most_frequent,1.0,1.0,1.000,1.000
18,LightGBM,leave_one_out,mean,No_info,1.0,1.0,1.000,1.000
24,LightGBM,leave_one_out,median,No_info,1.0,1.0,1.000,1.000
30,LightGBM,leave_one_out,most_frequent,No_info,1.0,1.0,1.000,1.000
36,LightGBM,leave_one_out,mean,bfill,1.0,1.0,1.000,1.000
42,LightGBM,leave_one_out,median,bfill,1.0,1.0,1.000,1.000
48,LightGBM,leave_one_out,most_frequent,bfill,1.0,1.0,1.000,1.000
39,LightGBM,сatboost,mean,bfill,1.0,1.0,0.681,0.630


## Проблема: 
На обучении и тесте получаю идеальные предсказания от деревьев и их вариаций. Если на обучении это можно обхяснить тем,что модель просто запоминает выборку, то на тесте модель должна была показать плохой результат. 

При этом одна и та же модель с разным методом кодирования данных и заполнения пропусков может давать разные результаты. Пример указан выше в датафрейме result3. 

Так же ,если вывести отдельно Значения метрик качества для тех методов,которые показали 100% точность - цифры будут отличаться от тех,что в датафрейме. См пример ниже

In [34]:
train_data_exp = train_data.copy()

train_data_exp[train_data_exp.columns.difference(train_data_exp.loc[:,'Var191':'Var229'].columns)]=fill_numeric(
    train_data_exp[train_data_exp.columns.difference(train_data_exp.loc[:,'Var191':'Var229'].columns)],
    strategy=num_fill_methods[0])

train_data_exp[train_data_exp.columns.difference(train_data_exp.loc[:,'Var1':'Var190'].columns)]=fill_cat(
    train_data_exp[train_data_exp.columns.difference(train_data_exp.loc[:,'Var1':'Var190'].columns)],
    strategy=cat_fill_methods[0])

train_data_exp=cat_coding(train_data_exp,  method=cat_code_methods[0])

print(models[3],cat_code_methods[3],num_fill_methods[0],cat_fill_methods[0])
results_append (model=models[3], cat_cod=cat_code_methods[3], fill_num=num_fill_methods[0],
                fill_cat=cat_fill_methods[0], data=train_data_exp)

kek
LightGBM leave_one_out mean most_frequent
LightGBM
Index(['Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6', 'Var7', 'Var9', 'Var10',
       'Var11',
       ...
       'Var220', 'Var221', 'Var222', 'Var223', 'Var224', 'Var225', 'Var226',
       'Var227', 'Var228', 'Var229'],
      dtype='object', length=212)
LGBMClassifier()
(1927, 212)
(827, 212)
Train auc score 1.0
Train f1 score 0.998
Test auc score 0.676
Test f1 score 0.628


#### Стоит сделать доп тесты на методы и найти ошибки. 